In [1]:
# default_exp losses
# all_slow

# Loss functions

> Various loss functions in PyTorch

In [2]:
# hide
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# export
import torch
import torch.nn as nn
import torch.nn.functional as F
from fastai.torch_core import TensorBase
from fastai.losses import *
from fastai.callback.core import Callback

In [4]:
from fastai.vision.all import *
import numpy as np
from torch.nn.modules.loss import _Loss
import segmentation_models_pytorch as smp
from steel_segmentation.utils import get_train_df
from steel_segmentation.transforms import SteelDataBlock, SteelDataLoaders

In [5]:
path = Path("../data")
train_pivot = get_train_df(path=path, pivot=True)
block = SteelDataBlock(path)
dls = SteelDataLoaders(block, train_pivot, bs=8)
xb, yb = dls.one_batch()
print(xb.shape, xb.device)
print(yb.shape, yb.device)

C:\Users\beanTech\miniconda3\envs\steel_segmentation\lib\site-packages\torch\_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ..\aten\src\ATen\native\BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


torch.Size([8, 3, 224, 1568]) cuda:0
torch.Size([8, 4, 224, 1568]) cpu


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [7]:
model = smp.Unet("resnet18", classes=4).to(device)

logits = model(xb)
probs = torch.sigmoid(logits)
preds = ( probs > 0.5).float()

C:\Users\beanTech\miniconda3\envs\steel_segmentation\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [8]:
# export
#reference: https://github.com/asanakoy/kaggle_carvana_segmentation/blob/master/asanakoy/losses.py
class SoftDiceLoss(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, logits, labels):
        probs = torch.sigmoid(logits)
        num = labels.size(0)
        m1 = probs.view(num, -1)
        m2 = labels.view(num, -1)
        intersection = (m1 * m2)
        score = 2. * (intersection.sum(1) + 1) / (m1.sum(1) + m2.sum(1) + 1)
        score = 1 - score.sum() / num
        return score

In [9]:
criterion = SoftDiceLoss()
criterion(logits.detach().cpu(), yb)

TensorImage(0.9883)

In [10]:
# export
#reference: https://github.com/zdaiot/Kaggle-Steel-Defect-Detection
class WeightedSoftDiceLoss(nn.Module):

    def __init__(self, size_average=True, weight=[0.2, 0.8]):
        super().__init__()
        self.size_average = size_average
        self.weight = torch.FloatTensor(weight)

    def forward(self, logit_pixel, truth_pixel):
        batch_size = len(logit_pixel)
        logit = logit_pixel.view(batch_size, -1)
        truth = truth_pixel.view(batch_size, -1)
        assert(logit.shape == truth.shape)

        loss = self.soft_dice_criterion(logit, truth)

        if self.size_average:
            loss = loss.mean()
        return loss

    def soft_dice_criterion(self, logit, truth):
        batch_size = len(logit)
        probability = torch.sigmoid(logit)

        p = probability.view(batch_size, -1)
        t = truth.view(batch_size, -1)

        w = truth.detach()
        self.weight = self.weight.type_as(logit)
        w = w * (self.weight[1] - self.weight[0]) + self.weight[0]

        p = w * (p*2 - 1)  #convert to [0,1] --> [-1, 1]
        t = w * (t*2 - 1)

        intersection = (p * t).sum(-1)
        union =  (p * p).sum(-1) + (t * t).sum(-1)
        dice  = 1 - 2 * intersection/union

        loss = dice
        return loss

In [11]:
criterion = WeightedSoftDiceLoss()
criterion(logits.detach().cpu(), yb)

TensorMask(0.9471)

In [12]:
# export
#reference: https://github.com/zdaiot/Kaggle-Steel-Defect-Detection
class SoftBCEDiceLoss(nn.Module):

    def __init__(self, bce_pos_weight, size_average=True, dice_weights=[0.2, 0.8], loss_weights=[0.7, 0.3]):
        super().__init__()
        self.size_average = 'mean' if size_average else 'none'
        self.loss_weights = loss_weights
        self.bce_loss = nn.BCEWithLogitsLoss(
            reduction=self.size_average, 
            pos_weight=torch.tensor(bce_pos_weight)
        )
        self.softdiceloss = WeightedSoftDiceLoss(
            size_average=self.size_average, 
            weight=dice_weights
        )

    def forward(self, input, target):
        input, target = TensorBase(input).float(), TensorBase(target).float()
        bce_loss = self.bce_loss(input, target)
        soft_dice_loss = self.softdiceloss(input, target)
        loss = self.loss_weights[0] * bce_loss + self.loss_weights[1] * soft_dice_loss

        return loss

In [13]:
criterion = SoftBCEDiceLoss(bce_pos_weight=1.5)
criterion(logits.detach().cpu(), yb)

TensorBase(0.7872)

In [14]:
# export
#reference: https://github.com/zdaiot/Kaggle-Steel-Defect-Detection
class MultiClassesSoftBCEDiceLoss(nn.Module):

    def __init__(self, classes_num=4, size_average=True, dice_weights=[0.2, 0.8], bce_pos_weights=[2.0,2.0,1.0,1.5], loss_weights=[0.7, 0.3], thresh=0.5):
        super().__init__()
        self.thresh = thresh
        self.classes_num = classes_num
        self.soft_bce_dice_losses = [
            SoftBCEDiceLoss(bce_pos_weight=pos_weight, size_average=size_average, dice_weights=dice_weights, loss_weights=loss_weights)
            for pos_weight in bce_pos_weights
        ]

    def forward(self, input, target):
        """
        Args:
            input: tensor, [batch_size, classes_num, height, width]
            target: tensor, [batch_size, classes_num, height, width]
        """
        loss = 0
        for class_index in range(self.classes_num):
            input_single_class = input[:, class_index, :, :]
            target_singlt_class = target[:, class_index, :, :]
            single_class_loss = self.soft_bce_dice_losses[class_index](input_single_class, target_singlt_class)
            loss += single_class_loss

        loss /= self.classes_num

        return loss

    def decodes(self, x):    
        return (x>self.thresh).float().argmax(dim=1)

    def activation(self, x): 
        return torch.sigmoid(x)

In [18]:
criterion = MultiClassesSoftBCEDiceLoss()
loss = criterion(logits.detach().cpu(), yb)
loss

TensorBase(0.7833)

In [20]:
criterion.decodes(logits.detach().cpu())

torch.Size([8, 224, 1568])

In [23]:
criterion.activation(logits.detach().cpu()).shape

torch.Size([8, 4, 224, 1568])

For the Tensorboard callback we need this Learner Callback to handle the step after the prediction.

In [16]:
# export
class LossEnabler(Callback):
    """Cast predictions and labels to TensorBase to compute the smp.losses"""
    def after_pred(self):
        if len(self.learn.yb) > 0:
            yb = self.learn.yb[0]
            self.learn.yb = (TensorBase(yb), )
            self.learn.pred = TensorBase(self.learn.pred)

In [17]:
dls.valid.bs 

8